# core

> Example chain to convert a snippet of Java EE code to Quarkus

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_experimental.utilities.python import PythonREPL

template = """You are an excellent enterprise architect who has an extensive
background in helping companies rewrite their legacy Java EE applications to Quarkus.

You will read a user's problem along with examples of how they have solved a problem in the past.
The past examples will be presented in format of a summary of the issue along with source code of 
that point in time along with the updated source code when the problem is fixed

You will then write Quarkus code to solve their current problem.
You will output the results in the form a diff which can be applied via 'git apply'.

Example #1 Issue: {example1_issue}

Example #1 Original Source Code:
{example1_original_code}

Example #1 Solved Source Code:
{example1_solved_code}

Current Issue: 
{current_issue}

Current Issue Original Source Code: 
{current_issue_original_code}

Your job is to look at the 'Current Issue' and the 'Current Issue Original Source Code' 
and rewrite the 'Current Issue Original Source Code' so the 'Current Issue' is solved 
in a manner similar to how 'Example #1 Original Source Code' was rewritten to 
'Example #1 Solved Source Code' 

Think through the changes you will make and explain each step of the process.
If you are unsure of what changes is needed please state you are unsure and ask 
for clarification to help you.

When you are done explaining the reasoning for each change, write the updated 
Quarkus source code in the form of a diff which can be applied via 'git apply' 
in Markdown format, e.g.:

```java
....
```"""

def _sanitize_output(text: str):
    return text
    #_, after = text.split("```java")
    #return after.split("```")[0]

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-16k")

chain = prompt | model | StrOutputParser() | _sanitize_output 


Turn debug on so we can see more of what langchain is doing

In [ ]:
from langchain.globals import set_debug
set_debug(False)

Let's get an example together from a Java EE app that has been migrated to Quarkus

We will be using the repo from:  
* https://github.com/mathianasj/eap-coolstore-monolith/tree/quarkus-migration

There is a script to help with fetching the repo at: 
* https://github.com/jwmatthews/kyma_poc/blob/main/data/fetch.sh


In [ ]:
from kyma_poc.scm import GitDiff

# We are taking 2 similar files in this repo and will using them as an example 
# to see quality of diff we can get from the LLM
example_git_path = "../data/coolstuff-quarkus"
gd = GitDiff(example_git_path)

example_1_file = "src/main/java/com/redhat/coolstore/service/OrderServiceMDB.java"
current_file = "src/main/java/com/redhat/coolstore/service/InventoryNotificationMDB.java"

example_1_file_commits = gd.get_commits_for_file(example_1_file)
current_file_commits = gd.get_commits_for_file(current_file)

example_1_original = gd.get_file_contents(example_1_file, example_1_file_commits[-1])
example_1_solved = gd.get_file_contents(example_1_file, example_1_file_commits[0])
current_original = gd.get_file_contents(current_file, example_1_file_commits[-1])

expected_diff = gd.get_patch_for_file(example_1_file_commits[-1], example_1_file_commits[0], example_1_file)



Lets run the chain and see it work

In [ ]:
result = chain.invoke({
    "example1_issue": "Something",
    "example1_original_code": example_1_original,
    "example1_solved_code": example_1_solved,
    "current_issue": "Something",
    "current_issue_original_code": current_original
    })

print(f"{result}")

The current issue is not clearly stated, so it is difficult to determine the exact problem that needs to be solved. However, based on the provided code, it seems that the issue is related to receiving and processing messages from a JMS topic.

To solve this issue, we can rewrite the code using Quarkus and the MicroProfile Reactive Messaging API. Here are the steps to make the necessary changes:

1. Update the package and import statements to use the appropriate Quarkus and MicroProfile annotations and classes.

2. Remove the unnecessary imports and dependencies related to JMS and WebLogic.

3. Annotate the class with `@ApplicationScoped` to make it a CDI bean.

4. Remove the `init()` and `close()` methods, as they are not needed in the Quarkus and MicroProfile Reactive Messaging context.

5. Replace the `onMessage()` method with a new method annotated with `@Incoming("inventory-incoming")` to specify the incoming channel name.

6. Change the method signature to accept a `String` parame